In [3]:
from datasets import load_dataset

SexComment = load_dataset("csv", data_files="SexComment.csv")['train']
SexComment_dataset = SexComment.remove_columns(['index','weibo_id','gender','location','like','date'])
SexComment_dataset = SexComment_dataset.rename_column('comment_text','text')
SexComment_dataset = SexComment_dataset.rename_column('label','label_sexist')
SexComment_dataset = SexComment_dataset.rename_column('text','text_cn')

 
dataset = SexComment_dataset.class_encode_column("label_sexist")

## remove long rows with long text 
dataset = dataset.filter(lambda x: len(x['text_cn'])<200)


In [4]:
import torch
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(model="Helsinki-NLP/opus-mt-zh-en", device=device)
# pipe = pipeline(model="Helsinki-NLP/opus-mt-zh-en")


def trans_CN_to_EN(cn: str) -> str:
    return pipe(cn)[0]['translation_text']

def add_text(row):
    row['text'] = trans_CN_to_EN(row['text_cn'])
    return row


In [3]:
## process token

# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

# def tokenization(example):
#     return tokenizer(example["text_cn"])

# dataset = dataset.map(tokenization, batched=True)

In [5]:
## Translate the dataset
dataset = dataset.map(add_text)


## Split the dataset

from datasets import ClassLabel, Value

new_features = dataset.features.copy()
new_features['label_sexist'] = ClassLabel(names=["not sexist", "sexist"])
new_dataset = dataset.cast(new_features)

split_dataset = new_dataset.train_test_split(0.1,stratify_by_column='label_sexist')
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']
split_again = train_dataset.train_test_split(0.1,stratify_by_column='label_sexist')
train_dataset = split_again['train']
validation_dataset = split_again['test']

## combine the dataset

from datasets import DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset,
})
print(dataset_dict)


## Upload the dataset

import json

file_path = "credentials.json"

with open(file_path, 'r') as json_file:
    token_data = json.load(json_file)

token = token_data.get("huggingface_token")
if token:
    print(f"Token read successfully")
else:
    print("Error: Token not found in the JSON file.")
    
import huggingface_hub

huggingface_hub.login(token)

dataset_dict.push_to_hub("yangezheng/SWSR-SexComment")

Map:   0%|          | 0/8571 [00:00<?, ? examples/s]

/home/yzheng/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Casting the dataset:   0%|          | 0/8571 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text_cn', 'label_sexist', 'category', 'target', 'text'],
        num_rows: 6941
    })
    validation: Dataset({
        features: ['text_cn', 'label_sexist', 'category', 'target', 'text'],
        num_rows: 772
    })
    test: Dataset({
        features: ['text_cn', 'label_sexist', 'category', 'target', 'text'],
        num_rows: 858
    })
})
Token read successfully
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/yzheng/.cache/huggingface/token
Login successful


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
import json

file_path = "credentials.json"

with open(file_path, 'r') as json_file:
    token_data = json.load(json_file)

token = token_data.get("huggingface_token")
if token:
    print(f"Token read successfully")
else:
    print("Error: Token not found in the JSON file.")
    
import huggingface_hub

huggingface_hub.login(token)

dataset_dict.push_to_hub("yangezheng/CMSB")